# Análisis de tópicos

En esta notebook vamos a descomponer en tópicos un conjunto de textos. La descomposición en tópicos pueden pensarla a la vez como un proceso de reducción dimensional, al describir los datos en el espacio de tópicos en vez del espacio original de features), o como un proceso de *clustering*, al agrupar los textos en dichos tópicos.

Vamos a aplicar dos algoritmos (NMF y LDA) sobre el dataset de libros de Gabriel García Márquez visto en clases anteriores.

In [1]:
# Importamos las librerías habituales
import pandas as pd 

import matplotlib.pylab as plt
import numpy as np

In [3]:
# Importamos nltk para extraer stopwords 
import nltk 
nltk.download('stopwords')

# Librería para hacer wordclouds
#from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Objetos de sklearn para hacer tópicos
from sklearn.feature_extraction.text import CountVectorizer # Contador de frecuencia
from sklearn.feature_extraction.text import TfidfTransformer # Creador de tf-idf

# Algoritmos de descomposición de tópicos
from sklearn.decomposition import NMF 
from sklearn.decomposition import LatentDirichletAllocation

Cargamos el dataset con el que vamos a trabajar desde la carpeta de la materia. Vamos a utilizar el dataset de la clase de sentiment sobre letras de distintos ritmos musicales:

In [11]:
# Cargamos el dataframe que generamos previamente
data_corpus = pd.read_pickle(r"C:\Users\HP\Documents\Ciencia De Datos\Análisis de los funerales de la mamá grande\corpus.pkl")
data_corpus.head()
text=[]
for i in range(0, len(data_corpus.Cuentos.keys())):
    text.append(data_corpus.Cuentos[i])

# Pasar el dataFrame a una estructura de filas y columnas 
data_corpus={'title':data_corpus.Cuentos.keys(), 
             'text':text
             }
data_corpus=pd.DataFrame(data_corpus)

### Construcción de la matriz documentos-términos

Vamos a construir esta matriz con valorización tf-idf, es decir, además de la frecuencia del término vamos a ponderar la especificidad. 
Esto lo hacemos en dos pasos: primero, describimos nuestros datos mediante frecuencia de términos; luego, le agregamos la valorización de la especificidad. 
Vamos además a remover las *stopwords* obtenidas de *nltk*.

In [13]:
# Lista de stopwords
stopwords = nltk.corpus.stopwords.words('spanish')

# Creamos el objeto contador de palabras, pidiéndole que remueve
# las stopwords, los términos que aparecen en un único documento (min_df)
# y los términos que aparecen en más del 70% de los documentos (max_df).
# Esto es para eliminar palabras raras (o errores de tipeo) y 
# términos que seguramente son stopwords no incluídos en la lista
count = CountVectorizer(min_df = 2, max_df = 0.70, stop_words = stopwords)

# Ajustamos con los datos. Acá especificamente creamos una matriz documentos-términos
x_count = count.fit_transform(data_corpus['text'])

# Dimensions de la matriz doc-tér
print(x_count.shape)

(10, 2253)


Podemos tranquilamente trabajar sobre la matriz de documentos descritos a través de la frecuencia de las palabras o bien, como es habitual, ponderar la especificidad de los términos mediante tf-idf. Esto lo hacemos de la siguiente manera:

In [14]:
# Creamos el objeto tf-idf. Le decimos además que devuelva los
# vectores documento con norma euclídea igual a 1 (norm = 'l2')
tfidf = TfidfTransformer(norm = 'l2')

# Creamos la matriz tf-idf a partir de la matriz de frecuencias
x_tfidf = tfidf.fit_transform(x_count)

### Aplicación de NMF sobre el corpus

Vamos a buscar los tópicos en nuestro corpus de textos a través de la descomposición en matrices no-negativas. Elijamos por ejemplo 5 tópicos:

In [15]:
# Elijamos la cantidad de tópicos
n_components = 5

# Construímos el objeto NMF con los tópicos indicados 
nmf = NMF(n_components = n_components)

# Aplicamos sobre nuestros datos
x_nmf = nmf.fit_transform(x_tfidf)

# Dimensión de la matriz transformada
print(x_nmf.shape)

(10, 5)


Veamos qué significa cada tópico. Lo que hay que indentificar es cuáles son los índices con mayor peso en cada componente y a qué término le corresponde. Para ello primero vamos a invertir el diccionario de vocabulario (para obtener otro del estilo "índice: término") y ordenar los índices de mayor a menor en cada componente:

In [16]:
# Objeto índice: término de nuestro vocabulario
vocabulary = {item: key for key, item in count.vocabulary_.items()}

# Para cada componente
for n in range(n_components):

  # Ordenamos una lista del largo de nuestro vocabulario según el peso en cada componente y nos quedamos con los primeros 10
  list_sorted = sorted(range(nmf.components_.shape[1]), reverse = True, key = lambda x: nmf.components_[n][x])[:10]

  # Printeamos los términos asociados a los valores más grande de cada una de las componentes
  print(', '.join([vocabulary[i] for i in list_sorted]))
  print('\n')

arcadio, josé, úrsula, buendía, melquíades, aldea, gitanos, aureliano, laboratorio, hombres


niña, tren, pueblo, padre, pájaros, madre, sacerdote, calor, estación, señora


montiel, jaula, josé, viuda, médico, señor, úrsula, niño, ricos, esposa


alcalde, café, gaveta, muela, viernes, abuela, junto, silla, sacó, ratones


mamá, reina, macondo, antonio, isabel, autoridad, aquella, padre, siglo, familia




Para que sea más fácil visualizar, armemos wordclouds, poniendole el peso dado por el algoritmo de NMF:

In [ ]:
# WordClouds
wc_atributos = {'height' : 800,
                'width' : 1200,
                'background_color' : 'white',
                'max_words' : 20
                } # Defino los parámetros que les voy a pasar a los wordclouds

# Creo la figura
fig, axs = plt.subplots(n_components, figsize = (6,20))

# Recorro para todas las componentes
for n in range(n_components):

  # 10 términos más pesados
  list_sorted = sorted(range(len(vocabulary)), reverse = True, key = lambda x: nmf.components_[n][x])[:10]

  # Diccionario término: peso
  comp_dict = {vocabulary[i]: nmf.components_[n][i] for i in list_sorted}

  # Creo el wordlcoud
  wc = WordCloud(**wc_atributos # De esta forma, le estoy diciendo a la función que expanda el diccionario de atributos de forma tal de que entienda lo que quiero que haga
                 ).generate_from_frequencies(comp_dict)

  axs[n].set_title('Tópico {}'.format(n))
  axs[n].imshow(wc)
  axs[n].axis('off')

plt.show()

#### Evolución de los tópicos

Algo que podemos hacer al identificar los tópicos es ver cómo evolucionan en el tiempo, es decir, si hay tópicos dominantes en determinados períodos. Antes de hacer esto, normalizemos los vectores documentos en el espacio de tópicos para interpretarlos como una distribución:

In [ ]:
# Normalizador
from sklearn.preprocessing import Normalizer 

# Creamos un objeto para normalizar a que la suma dé 1
norm = Normalizer('l1')

# Sobreescribimos sobre la matriz de documentos-tópicos
x_nmf = norm.fit_transform(x_nmf)

# Guardemos en el dataframe esta información
for n in range(n_components):
  df['nmf_comp{}'.format(n)] = x_nmf[:,n]

df.head()